<a href="https://colab.research.google.com/github/Cerabbite/CGEL/blob/main/Word.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For each word in the top 10.000 most used English, Spanish, Japanese and Chinese words. There will be 100-1.000 sample sentences with those words (written in using the english alphabet). Using these sentences this script is going to work out how related words are.

In [2]:
# Import libraries
#!pip install keras
import tensorflow as tf
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Embedding, LSTM, Dense, Bidirectional
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.models import Sequential
from tensorflow.keras.optimizers import Adam
import numpy as np 

In [3]:
tokenizer = Tokenizer()

data = open("irish poetry.txt").read()
corpus = data.lower().split("\n")

tokenizer.fit_on_texts(corpus)
total_words = len(tokenizer.word_index) + 1

In [4]:
input_sequences = []
for line in corpus:
  token_list = tokenizer.texts_to_sequences([line])[0]
  for i in range(1, len(token_list)):
    n_gram_sequence = token_list[:i+1]
    input_sequences.append(n_gram_sequence)

In [5]:
max_sequence_len = max([len(x) for x in input_sequences])
input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='pre'))

In [6]:
xs = input_sequences[:,:-1]
labels = input_sequences[:,-1]

ys = tf.keras.utils.to_categorical(labels, num_classes=total_words)

In [8]:
model = Sequential()
model.add(Embedding(total_words, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(150)))
model.add(Dense(total_words, activation='softmax'))
adam = Adam(lr=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
history = model.fit(xs, ys, epochs=20, verbose=1)


Epoch 1/20


/usr/local/lib/python3.7/dist-packages/keras/optimizer_v2/adam.py:105: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super(Adam, self).__init__(name, **kwargs)


377/377 [==============================] - 31s 72ms/step - loss: 6.6547 - accuracy: 0.0729
Epoch 2/20
377/377 [==============================] - 27s 71ms/step - loss: 5.7625 - accuracy: 0.1133
Epoch 3/20
377/377 [==============================] - 27s 71ms/step - loss: 4.9098 - accuracy: 0.1679
Epoch 4/20
377/377 [==============================] - 27s 71ms/step - loss: 4.0198 - accuracy: 0.2311
Epoch 5/20
377/377 [==============================] - 27s 71ms/step - loss: 3.1897 - accuracy: 0.3360
Epoch 6/20
377/377 [==============================] - 27s 72ms/step - loss: 2.5242 - accuracy: 0.4308
Epoch 7/20
377/377 [==============================] - 27s 72ms/step - loss: 2.0318 - accuracy: 0.5293
Epoch 8/20
377/377 [==============================] - 27s 72ms/step - loss: 1.6885 - accuracy: 0.5993
Epoch 9/20
377/377 [==============================] - 27s 72ms/step - loss: 1.4148 - accuracy: 0.6659
Epoch 10/20
377/377 [==============================] - 28s 74ms/step - loss: 1.2598 - accurac

In [12]:
# Test the model
seed_text = "I made a poetry machine"
next_words = 10

for _ in range(next_words):
  token_list = tokenizer.texts_to_sequences([seed_text])[0]
  token_list = pad_sequences([token_list], maxlen=max_sequence_len-1, padding='pre')
  predicted = model.predict(token_list, verbose=0)
  classes = np.argmax(predicted,axis=1)
  output_word = ""
  for word, index in tokenizer.word_index.items():
    if index == classes:
      output_word = word
      break
  seed_text += f" {output_word}"

print(seed_text)


I made a poetry machine guns for erin go bragh the same dressed the roses
